for misc testing

In [1]:
import sys; sys.path.append("../models")
import utils
import train
import MLP

import torch as t
import torch.nn as nn
import torch.nn.functional as F

import xarray as xr

t.manual_seed(123456)
device = "cuda" if t.cuda.is_available() else "cpu"

In [2]:
# note: the aim is for all preprocessing / reshaping to be folded into SimDataset

data_home = "/mnt/g/My Drive/GTC/solodoch_data_minimal"
lats = ["26N", "30S", "55S", "60S"]

lat = lats[0]
data = xr.open_dataset(f"{data_home}/{lat}.nc")

# apply whatever preprocessing we want *before* calling reshape_inputs
pp_data = utils.apply_preprocessing(data,
                                    mode="inputs",
                                    remove_season=True,
                                    remove_trend=True,
                                    standardize=True,
                                    lowpass=True)

# reshape as desired and convert to a tensor
pp_data = utils.reshape_inputs(pp_data, keep_coords=["time", "longitude"], history=6, return_pt=True)

# dummy strength data
strength = t.rand(pp_data.shape[0])
X = pp_data; y = strength

axes: ['time', 'history', 'longitude', 'feature']
variables: ['SSH', 'SST', 'SSS', 'OBP', 'ZWS']
shape: (283, 6, 164, 5)


In [3]:
X = X.reshape(X.shape[0], -1)
m = MLP.MLP(X.shape[1], 2, 16, 0.2)
out = m(X)
print(out.shape)

torch.Size([283, 1])
